In [1]:
import pysam
import pandas as pd
import numpy as np
import sys
from __future__ import print_function
import cPickle

In [2]:
with open("../reads/meta/s2.tsv") as f:
    data = pd.read_table(f).set_index('EMBL ID').drop(["Strain/species details"], axis=1).to_dict()["Phylum"]
id2phlm = {}
for k,v in data.items():
    nk = k.split(".")[0]
    id2phlm[nk] = v
    if (nk == "CM000636"):
        id2phlm["CP006835"] = v
del data

In [ ]:
mil = 1000000
hund = 100

def get_ref_id(aln):
    # get id for the mapped reference
    try:
        return aln.reference_name.split("|")[1]
    except:
        return aln.reference_name.split(".")[0]
    
def print_details(qId, rId, aln):
    print ("ERROR")
    print("SAM Algns")
    print (aln.query_name, aln.reference_name)
    print("SUBSETTING")
    print (qId, rId)
    print ("PHYLA")
    print (id2phlm[qId], id2phlm[rId])


def get_stats(fname, rname, level="phyla"):
    totReads = 0
    TN = 0
    FN = 0
    reads_list = []
    roseCount = 0
    euCount = 0
    with open(rname) as f:        
        for line in f:
            # counting total reads
            totReads += 1
            # Progress Monitoring
            if(totReads % mil == 0):
                print ("\r Done reading {} Million reads.".format(int(round(totReads)/1000000)), end="")
            
            #extracting relevant part of read
            read = line.strip().replace("/1","").replace("@","")
            
            if "Random" in read:
                TN += 1
            if "Eukaryotes" in read:
                euCount += 1
            if "Rose" in read:
                roseCount += 1
            
            #making a list of read id
            reads_list.append(read)
            
            # skip next 4 lines
            for _ in range(3):
                f.next()
    
    if len(reads_list) != len(set(reads_list)):
        print ("ERROR: Repeating reads found")
        return 0

# ++++++++++++++++++++++++++++++++++++++++++++++++++++
# Repeating Block
# ++++++++++++++++++++++++++++++++++++++++++++++++++++
#     print("\nSaving Pickle")
#     with open(r"reads_list.pickle", "wb") as f:
#         cPickle.dump(reads_list, f)
#     print("Done Saving Pickle")
# ++++++++++++++++++++++++++++++++++++++++++++++++++++

# ++++++++++++++++++++++++++++++++++++++++++++++++++++
# Repeating Block
# ++++++++++++++++++++++++++++++++++++++++++++++++++++
#     print("Reading Pickle")
#     with open(r"reads_list.pickle", "rb") as f:
#         reads_list = cPickle.load(f)
#     totReads = len(reads_list)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++
    
    with pysam.AlignmentFile(fname) as f:
        TP = 0
        FP = 0
        totCount = 0.0
        singCount = 0
        orphanCount = 0
        skipCount = 0.0
        for aln in f:
            #get mate of the read
            mate_aln = f.next()
            
            # count total Number of reads
            totCount += 1
            
            # get number of alignments
            n_alns = aln.get_tag('NH')
            
            #ignoring Rose Sequence
            if "Rose" in aln.query_name or "Eukaryotes" in aln.query_name:
                skipCount += 1.0/n_alns
                continue
            
            # Ignoring Orphan alignments for now
            if(aln.reference_name != mate_aln.reference_name):
                orphanCount += 1
                print ("WARNING: ORPHANS Detected statistics Neess to be re-evaluated")
                continue
            
            # Progress Monitoring
            if(round(totCount) % mil == 0):
                print ("\r Done reading {} Million reads.".format(int(round(totCount)/1000000)), end="")

            # get ground truth id
            qId = aln.query_name.split('-')[0]
            if "|" in qId:
                qId = qId.split("|")[1]
            elif "_" in qId:
                qId = qId.split("_")[0]
                
            # for singly mapped reads only
            if n_alns == 1:
                # Increment the single count
                singCount += 1

                # get id for the mapped reference
                rId = get_ref_id(aln)
                
                # compare the labels
                try:
                    if qId == rId or id2phlm[qId]==id2phlm[rId]:
                        # get TP count 
                        TP += 1
                    else:
#                         print_details(qId, rId, aln)
                        # get FP count 
                        FP += 1
                except:
                    print ("In Single mapping")
                    print_details(qId, rId, aln)

                    
            #handling MultiMapped Reads
            else:
                # list of all alignments
                algns = [get_ref_id(aln)]
                
                # iterate over all alignments
                for _ in range(1, n_alns):
                    aln = f.next()
                    mate_aln = f.next()
                    
                    # Ignoring Orphan alignments for now
                    if(aln.reference_name != mate_aln.reference_name):
                        orphanCount += 1
                    else:
                        algns.append(get_ref_id(aln))
                
                # SIMULATED FALSE POSITIVES
                if "Random" in aln.query_name:
                    TN -= 1
                    FP += 1
                    continue
                
                flag = False
                for rId in algns:
                    try:
                        if qId == rId or id2phlm[qId]==id2phlm[rId]:
                            flag = True
                            break
                    except:
                        print ("In Multimapping")
                        print_details(qId, rId, aln)

                if flag:
                    TP += 1
                else:
#                     print_details(qId, rId, aln)
                    FP += 1
    
    mmCount = round(totCount - singCount)
    unmapCount = totReads - totCount
    FN = totReads - TP - FP - TN - roseCount - euCount
    print ("\n")
    print ("====================================================================================")
    print ("Total Number of reads: {0} ({1:.2f}M)".format(totReads, totReads/mil))    
    print ("Number of Unmapped reads: {0} ({1:.2f}M, {2:.2f}%)".format(unmapCount, unmapCount/mil, unmapCount*hund/totReads))    
    print ("Number of Mapped reads {0}({1:.2f}M, {2:.2f}%)".format(totCount, totCount/mil, totCount*hund/totReads))
    print ("\n")
    print ("============================ OUT OF MAPPED READS ===================================")
    print ("Number of Singly Mapped reads: {0} ({1:.2f}M, {2:.2f}%)".format(singCount, singCount/mil, singCount*hund/totReads))
    print ("Number of Multimapped reads: {0} ({1:.2f}M, {2:.2f}%)".format(mmCount, mmCount/mil, mmCount*hund/totReads))
    print ("Number of Mapped but Skipped reads: {0} ({1:.2f}M, {2:.2f}%)".format(skipCount, skipCount/mil, skipCount*hund/totReads))
    print ("Number of Orphaned (Ignored)ALIGNMENTS (Should be significantly low): {}".format(orphanCount))
    print ("====================================================================================")
    print ("\n ===================== \n ASSUMPTIONS \n =====================")
    print ("1: Any Multi-mapped read has the Original Phyla in ATLEAST 1 alignment")
    print ("2: Don't know what to do with Rose Sequence Ignoring for now")
    print ("3: No reference of Eukaryotes added")
    print ("OVERALL: Atmost 10% Reads could have been mapped more.")
    print ("Eukaryotes Counts: {0}({1:.2f}%)".format(euCount, euCount/totReads))
    print ("Rose Counts: {0}({1:.2f}%)".format(roseCount, roseCount/totReads))
    print ("====================================================================================")
    print ("\n ===================== \n ACCURACY METRIC \n =====================")
    print ("Number of True positives(TP) reads: {0} ({1:.2f}M, {2:.2f}%)".format(TP, TP/mil, TP*hund/totReads))
    print ("Number of False Negatives(FN) reads: {0} ({1:.2f}M, {2:.2f}%)".format(FN, FN/mil, FN*hund/totReads))
    print ("Number of False positives(FP) reads: {0} ({1:.2f}M, {2:.2f}%)".format(FP, FP/mil, FP*hund/totReads))
    print ("Number of True Negatives(TN) reads: {0} ({1:.2f}M, {2:.2f}%)".format(TN, TN/mil, TN*hund/totReads))
    print ("====================================================================================")
            
get_stats("../bam/A1.sam", "../reads/A1_1.fastq")

 Done reading 6 Million reads..